In [1]:
import numpy as np
import pandas as pd

import anndata
import scanpy as sc
from ALLCools.mcds import MCDS
from wmb import aibs


In [2]:
aibs_annot = aibs.get_tenx_annot()
aibs_annot

<xarray.AIBSTENXCellAnnotation>
Dimensions:           (cell: 4065284, L1_cat: 12, L2_cat: 306)
Coordinates:
  * cell              (cell) object 'L8TX_180406_01_B01-TCTTCGGCAGCCTTTC-1' ....
  * L1_cat            (L1_cat) int64 0 1 2 3 4 5 6 7 8 9 10 11
  * L2_cat            (L2_cat) int64 0 1 2 3 4 5 6 ... 300 301 302 303 304 305
Data variables:
    L1                (cell) int8 dask.array<chunksize=(4065284,), meta=np.ndarray>
    L1_cat_annot      (L1_cat) object dask.array<chunksize=(12,), meta=np.ndarray>
    L2                (cell) int16 dask.array<chunksize=(4065284,), meta=np.ndarray>
    L2_cat_annot      (L2_cat) object dask.array<chunksize=(306,), meta=np.ndarray>
    L3                (cell) uint32 dask.array<chunksize=(4065284,), meta=np.ndarray>
    sample            (cell) <U18 dask.array<chunksize=(4065284,), meta=np.ndarray>
    L1_annot          (cell) object 'Glut' 'Glut' 'Glut' ... 'Glut' 'Glut'
    L2_annot          (cell) object 'L2 IT PPP-APr Glut' ... 'APN C1ql4 Glut'
    DissectionRegion  (cell) object 'PAR-POST-PRE-SUB-ProS' ... 'MB - MRN-CUN...
    MajorRegion       (cell) object 'HPF' 'HPF' 'HPF' 'HPF' ... 'MB' 'MB' 'MB'
    SubRegion         (cell) object 'RHP' 'RHP' 'RHP' 'RHP' ... 'MB' 'MB' 'MB'
    L4                (cell) uint32 dask.array<chunksize=(4065284,), meta=np.ndarray>
Attributes:
    cluster_data_var:   ['L1', 'L2', 'L3']
    cluster_hierarchy:  [['L1'], ['L2'], ['L3']]
    annot_path:         /cemba/wmb/aibs/AIBS.TENX.Annotations.v3.zarr
    __cache__:          {}

In [3]:
annot = aibs_annot[['L1', 'L2', 'L3', 'L1_annot', 'L2_annot', 'DissectionRegion', 'SubRegion', 'MajorRegion']].to_pandas()

In [4]:
NN_types = ['NN', 'IMN']
nonoverlap_types = np.concatenate([annot['L2_annot'][annot['L2_annot'].str.startswith('CB')].unique(),
                                   annot['L2_annot'][annot['L2_annot'].str.startswith('DCO')].unique(),
                                   annot['L2_annot'][annot['L2_annot'].str.startswith('MM')].unique(),
                                  ])


In [5]:
rna_region_dict = {
    'CTX': ["RSP", "ACA","PL-ILA-ORB", "AI-CLA", "AId-AIv", "AId-AIv-AIp", "TEa-PERI-ECT", "AUD-TEa-PERI-ECT", "AUD", "PTLp", "VIS", "VIS-PTLp", "VISl", 'VISm', 'VISp', "MOp", "MOs-FRP", "MO-FRP", "SSp", "SSs-GU-VISC-AIp", "SS-GU-VISC"],
    'HIP': ["HIP", "HIP - CA"],
    'OLF': ["OLF - MOB-AOB", "OLF - AON-TT-DP"],
    'RHP': ["PAR-POST-PRE-SUB-ProS", 'ENT'],
    'PIR': ["OLF - COA-PAA-NLOT-TR","OLF - PIR", "OLF - AON-TT-DP-PIR-COA-NLOT-PAA-TR", 'CTXsp - CLA-EP-LA-BLA-BMA-PA'],
    'AMY': ["STR - sAMY"],
    'STR': ["STR - STRd", "STR - STRv"],
    'PAL': ["CNU - PAL"],
    'TH': ["TH - Middle-lateral", "TH - LGd-IGL-LGv", "TH - Posterior", "TH - Anterior", "TH - PVT-PT", "TH - MG-SPFp-PP-POL-SGN-PoT-PIL", "TH - PO-Eth", "TH - PF-SPA-SPFm-VPMpc-VPLpc-RT", "TH", "TH - MH-LH-LP","TH - Middle-medial", "TH - AD-AV-AM-IAD-LD", "TH - RE-RH-CM-IAM-SMT-PR-Xi", "TH - MD-IMD-PCN-CL", "TH - VAL-VPM-VPL-VM"], 
    'HY': ["HY - HYa1", "HY - HYa2", "HY - HYm2", "HY - HYml", "HY - HYmm", "HY - HYpl", "HY - HYpm", "HY - MEZ-PVZ-PVR", "HY LZ"], 
    'MB': ["MB - VTA-SN", "MB - IC", "MB - MRN-CUN-RN-RR-PPN-PRT-SC-IC","MB - SC-IC",'MB - SC',"MB - MRN-CUN-RN-RR-PN", "MB - MRN-CUN-RN-RR-PPN-NB-SAG-PBG", "MB - MRN-CUN-RN-RR-PPN-PRT", "MB - PAG-RAmb", 'MB - PRT'], 
    'HB': ["MY - AP-DCO-VCO-DCN-ECU-NTS", "MY - CN-DCN-ECU-NTS-SPVC-SPVI-SPVO", "MY - MYsat-MYmot", "MY - MYsat-MYmot ant", "MY - MYsat-MYmot post", "MY - MYsat-MYmot-vent post", "MY - MYsat-Mymot-vent ant", "MY - SPVC-SPVI-SPVO", "MY - VNC-PHY-DMX-XII","HB - MY", "PONS - Pmot-Psat ant", "PONS - Pmot-Psat post", "PONS - Psen"]
}


In [7]:
# Parameters
group_name = 'STR'


In [8]:
t = group_name
selregion = rna_region_dict[t]


In [9]:
# select all CB cells
selc = annot.loc[annot['DissectionRegion'].isin(selregion) & (~annot['L1_annot'].isin(NN_types)) & (~annot['L2_annot'].isin(nonoverlap_types))].index
len(selc)

41012

In [10]:
rnads_path = aibs.AIBS_TENX_V2_ZARR_PATH
rna_ds = MCDS.open(rnads_path, use_obs=selc, var_dim='gene')
rna_ds

<xarray.MCDS>
Dimensions:     (cell: 41012, gene: 32285)
Coordinates:
  * cell        (cell) <U37 'L8TX_201105_01_A02-CCGTTCAGTTTGACAC-1' ... 'L8TX...
  * gene        (gene) <U18 'ENSMUSG00000051951' ... 'ENSMUSG00000095041'
    gene_count  (cell) float32 6.662e+03 5.795e+03 ... 3.855e+03 2.012e+03
    gene_name   (gene) <U16 'Xkr4' 'Gm1992' ... 'AC234645.1' 'AC149090.1'
    umi_count   (cell) float32 3.057e+04 2.23e+04 ... 9.605e+03 4.554e+03
Data variables:
    gene_da     (cell, gene) float32 dask.array<chunksize=(5000, 32285), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  gene

In [11]:
cef = pd.read_csv(f'{t}_mC_CEF.txt', header=None, index_col=0).index
cef.name = 'gene'
cef

Index(['ENSMUSG00000025905', 'ENSMUSG00000033740', 'ENSMUSG00000079671',
       'ENSMUSG00000067879', 'ENSMUSG00000099032', 'ENSMUSG00000042501',
       'ENSMUSG00000048960', 'ENSMUSG00000097171', 'ENSMUSG00000016918',
       'ENSMUSG00000025938',
       ...
       'ENSMUSG00000041362', 'ENSMUSG00000006270', 'ENSMUSG00000025094',
       'ENSMUSG00000087095', 'ENSMUSG00000043969', 'ENSMUSG00000100119',
       'ENSMUSG00000118031', 'ENSMUSG00000118190', 'ENSMUSG00000057858',
       'ENSMUSG00000117790'],
      dtype='object', name='gene', length=6520)

In [12]:
gene_id_not_nan = ~rna_ds.get_index('gene').isna()
is_cef = rna_ds.get_index('gene').isin(cef)
use_genes = gene_id_not_nan & is_cef
use_genes.sum()

6520

In [13]:
adata = rna_ds.get_count_adata(da_name='gene_da', use_vars=use_genes)
adata

Loading chunk 0-10000/41012
Loading chunk 10000-20000/41012
Loading chunk 20000-30000/41012
Loading chunk 30000-40000/41012
Loading chunk 40000-41012/41012


AnnData object with n_obs × n_vars = 41012 × 6520
    obs: 'count', 'umi_count'
    var: 'name'

In [14]:
# same as sc.pp.normalize_per_cell(adata) but this method only normalize to 
# total read counts of the remaining CEF 
# instead of all the gene counts as saved in umi_count

adata.X.data = adata.X.data/np.repeat(adata.obs['umi_count'].values, adata.X.getnnz(axis=1)
                                     ) * adata.obs['umi_count'].mean()


In [15]:
sc.pp.log1p(adata)
#sc.pp.scale(adata) this method is supposed to use large memory so skip this 

In [16]:
adata.var['mean'] = adata.X.mean(axis=0).A1
adata.var['std'] = (adata.X.multiply(adata.X)).mean(axis=0).A1 - (adata.var['mean'].values ** 2)
print(adata.var['std'].min())


0.0


In [17]:
adata.obs[annot.columns] = annot.loc[adata.obs.index,:]
adata

AnnData object with n_obs × n_vars = 41012 × 6520
    obs: 'count', 'umi_count', 'L1', 'L2', 'L3', 'L1_annot', 'L2_annot', 'DissectionRegion', 'SubRegion', 'MajorRegion'
    var: 'name', 'mean', 'std'
    uns: 'log1p'

In [18]:
pass_std_filter = (adata.var['std'].values > 0.005)
adata._inplace_subset_var(pass_std_filter)


In [19]:
np.savetxt('use_genes.txt', adata.var.index, fmt='%s')


In [20]:
adata.write_h5ad(f'{t}_{adata.shape[0]}_rna.h5ad')
